# 1. Identifying Valid Participants, Converting Raw Filenames to Correct Naming Formats

In [31]:
import os
import shutil
from pathlib import Path
from typing import Iterable, List, Tuple, Dict
import importlib

from components import file_handling as fh
importlib.reload(fh)

# ===================================================
# IDENTIFICATION OF VALID PARTICIPANTS
# - Params: 
#   - `root_dir`: Directory path that contains all trial directories as immediate child folders.
#   - `expected_files`: List of filenames expected within each child folder
# - Returns:
#   - `valids`: list of directories that had all expected files
#   - `invalids`: list of directories that did not contain one or more expected files
def identify_valid_participants( root_dir:str, 
                                 expected_files:Iterable[str] ) -> Tuple[List[str], List[str]]:
    # Initialize paths
    root_path = Path(root_dir)
    expected = set(expected_files)

    # Initialize outputs
    with_all:List[str] = []
    missing:List[str] = []

    # Start iterating through all immediate subdirectories
    for subdir in root_path.iterdir():
        # Ignore if subdirectory is not a directory
        if not subdir.is_dir(): continue
        # Get present files in the subdirectory
        present = {p.name for p in subdir.iterdir() if p.is_file()}
        missing_files = sorted(expected - present)
        # Check if files are within subset of expected
        if expected.issubset(present):  with_all.append(str(subdir))
        else:                           missing.append((str(subdir), len(missing_files), missing_files))

    # Return both
    return with_all, missing

# COPY AND RENAME VALID PARTICIPANT FILES
# - Params:
#   - src_subdirs: a List of participant directories
#   - rename_map: a Dictionary dictating how to rename files
#   - dest_root: the parent directory where participant directories should be saved
# - Returns
#   - List of output participant directories
def copy_and_rename( src_subdirs: Iterable[str],
                     rename_map: Dict[str, str],
                     dest_root: str ) -> List[str]:

    # Initialize destination directory
    fh.mkdirs(dest_root)
    dest_root = Path(dest_root)

    # Initialize new subdirs list
    new_subdirs:List[str] = []

    # Iterate through existing source dirs
    for src in src_subdirs:
        # Prep copied dir
        src_dir = Path(src)
        dest_dir = dest_root / src_dir.name
        dest_dir.mkdir(parents=True, exist_ok=True)
        # Iterate through each file in each source directory
        for item in src_dir.iterdir():
            # Ignore any that are not files
            if not item.is_file(): continue
            # Remaps
            new_name = rename_map.get(item.name, item.name)
            dest_path = dest_dir / new_name
            # Copy
            shutil.copy2(item, dest_path)
        # save new output directory
        new_subdirs.append(str(dest_dir))
    # Return \
    return new_subdirs
# ===================================================

_RAWS_DIR = os.path.join('.', 'samples', 'raws')
_ROOT_DIR = os.path.join('.', 'samples', 'renamed')
_EXPECTED_FILES = {
    'calibration_test.csv':     'calibration_0.csv',
    'calibration_test_1.csv':   'calibration_1.csv',
    'calibration_test_2.csv':   'calibration_2.csv',
    'calibration_test_3.csv':   'calibration_3.csv',
    'calibration_test_4.csv':   'calibration_4.csv',
    'calibration_test_5.csv':   'calibration_5.csv',
    'calibration_test_6.csv':   'calibration_6.csv',
    'calibration_test_7.csv':   'calibration_7.csv',
    'eye.csv':                  'eye.csv',
    'eeg-rest.csv':             'eeg_rest.csv',
    'eeg-vr.csv':               'eeg_vr.csv',
    'pedestrians.csv':          'pedestrians.csv'
}
_TRIAL_NAMES = [
    "Trial-ApproachAudio Start", 
    "Trial-BehindAudio Start", 
    "Trial-Behind Start", 
    "Trial-AlleyRunnerAudio Start", 
    "Trial-AlleyRunner Start", 
    "Trial-Approach Start"
]

present, missing = prep_data(_RAWS_DIR, _EXPECTED_FILES.keys())
print("Present:")
for p in present: print(f'\t{p}')
print("Missing:")
for p in missing: print(f'\t{p}')

pdirs = copy_and_rename(present, _EXPECTED_FILES, _ROOT_DIR)
print("New Source Files:")
for p in pdirs: print(f'\t{p}')


Present:
	samples/raws/P3
	samples/raws/P2
	samples/raws/P5
	samples/raws/P4_Mus2
	samples/raws/P7
	samples/raws/P9
	samples/raws/P6
Missing:
	('samples/raws/PI1', 10, ['calibration_test.csv', 'calibration_test_1.csv', 'calibration_test_2.csv', 'calibration_test_3.csv', 'calibration_test_4.csv', 'calibration_test_5.csv', 'calibration_test_6.csv', 'calibration_test_7.csv', 'eeg-rest.csv', 'eeg-vr.csv'])
	('samples/raws/P10', 2, ['eeg-rest.csv', 'eeg-vr.csv'])
	('samples/raws/P0', 11, ['calibration_test.csv', 'calibration_test_1.csv', 'calibration_test_2.csv', 'calibration_test_3.csv', 'calibration_test_4.csv', 'calibration_test_5.csv', 'calibration_test_6.csv', 'calibration_test_7.csv', 'eeg-vr.csv', 'eye.csv', 'pedestrians.csv'])
	('samples/raws/PI2', 10, ['calibration_test.csv', 'calibration_test_1.csv', 'calibration_test_2.csv', 'calibration_test_3.csv', 'calibration_test_4.csv', 'calibration_test_5.csv', 'calibration_test_6.csv', 'calibration_test_7.csv', 'eeg-rest.csv', 'eeg-vr.csv

# 2. File Sorting into Trials

In [ ]:
from components import eeg_handling as eh
importlib.reload(eh)

# SEPARATE FILES INTO TRIALS
def separate_into_trials( pdirs: Iterable[str] ) -> None:
    # Iterate through eahch participant directory
    for pdir in pdirs:
        # The eeg_rest data 
        eeg, blinks = eh.read(os.path.join(pdir, 'eeg_rest.csv'))
        raw, processed = eeg_handling.separate(eeg)

separate_into_trials(pdirs)

,TimeStamp,Delta_TP9,Delta_AF7,Delta_AF8,Delta_TP10,Theta_TP9,Theta_AF7,Theta_AF8,Theta_TP10,Alpha_TP9,...,HSI_TP9,HSI_AF7,HSI_AF8,HSI_TP10,Battery,Elements,unix_ms,unix_sec,rel_sec,rel_ms
1,2025-10-01 15:28:53.758,1.353029,0.090971,0.568236,1.768482,1.233290,-0.239048,-0.248181,1.546042,1.372374,...,2.0,1.0,1.0,2.0,100.0,NaN,1759346933758,1.759347e+09,0.077,77
2,2025-10-01 15:28:53.759,1.353029,0.090971,0.568236,1.768482,1.233290,-0.239048,-0.248181,1.546042,1.372374,...,2.0,1.0,1.0,2.0,100.0,NaN,1759346933759,1.759347e+09,0.078,78
3,2025-10-01 15:28:53.759,1.353029,0.090971,0.568236,1.768482,1.233290,-0.239048,-0.248181,1.546042,1.372374,...,2.0,1.0,1.0,2.0,100.0,NaN,1759346933759,1.759347e+09,0.078,78
4,2025-10-01 15:28:53.760,1.353029,0.090971,0.568236,1.768482,1.233290,-0.239048,-0.248181,1.546042,1.372374,...,2.0,1.0,1.0,2.0,100.0,NaN,1759346933760,1.759347e+09,0.079,79
5,2025-10-01 15:28:53.760,1.353029,0.090971,0.568236,1.768482,1.233290,-0.239048,-0.248181,1.546042,1.372374,...,2.0,1.0,1.0,2.0,100.0,NaN,1759346933760,1.759347e+09,0.079,79
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11178,2025-10-01 15:29:36.402,-0.190751,0.090971,0.181605,1.838536,0.417479,-0.239048,-0.117757,0.627978,0.820460,...,1.0,2.0,1.0,1.0,100.0,NaN,1759346976402,1.759347e+09,42.721,42721
11179,2025-10-01 15:29:36.403,-0.190751,0.090971,0.181605,1.838536,0.417479,-0.239048,-0.117757,0.627978,0.820460,...,1.0,2.0,1.0,1.0,100.0,NaN,1759346976403,1.759347e+09,42.722,42722
11180,2025-10-01 15:29:36.403,-0.190751,0.090971,0.181605,1.838536,0.417479,-0.239048,-0.117757,0.627978,0.820460,...,1.0,2.0,1.0,1.0,100.0,NaN,1759346976403,1.759347e+09,42.722,42722
11181,2025-10-01 15:29:36.403,-0.190751,0.090971,0.181605,1.838536,0.417479,-0.239048,-0.117757,0.627978,0.820460,...,1.0,2.0,1.0,1.0,100.0,NaN,1759346976403,1.759347e+09,42.722,42722


,TimeStamp,unix_sec,unix_ms,rel_sec,rel_ms
1974,2025-10-01 15:29:00.518,1.759347e+09,1759346940518,6.837,6837
9257,2025-10-01 15:29:28.949,1.759347e+09,1759346968949,35.268,35268
10038,2025-10-01 15:29:31.997,1.759347e+09,1759346971997,38.316,38316
10195,2025-10-01 15:29:32.619,1.759347e+09,1759346972619,38.938,38938
10352,2025-10-01 15:29:33.218,1.759347e+09,1759346973218,39.537,39537
10847,2025-10-01 15:29:35.158,1.759347e+09,1759346975158,41.477,41477
10952,2025-10-01 15:29:35.528,1.759347e+09,1759346975528,41.847,41847
11109,2025-10-01 15:29:36.120,1.759347e+09,1759346976120,42.439,42439


,TimeStamp,Delta_TP9,Delta_AF7,Delta_AF8,Delta_TP10,Theta_TP9,Theta_AF7,Theta_AF8,Theta_TP10,Alpha_TP9,...,HSI_TP9,HSI_AF7,HSI_AF8,HSI_TP10,Battery,Elements,unix_ms,unix_sec,rel_sec,rel_ms
1,2025-09-25 15:37:14.751,0.092150,0.0,0.208309,0.324425,0.803366,0.0,0.117574,0.489226,1.779712,...,1.0,1.0,1.0,1.0,96.0,NaN,1758829034751,1.758829e+09,0.068,68
2,2025-09-25 15:37:14.752,0.092150,0.0,0.208309,0.324425,0.803366,0.0,0.117574,0.489226,1.779712,...,1.0,1.0,1.0,1.0,96.0,NaN,1758829034752,1.758829e+09,0.069,69
3,2025-09-25 15:37:14.752,0.092150,0.0,0.208309,0.324425,0.803366,0.0,0.117574,0.489226,1.779712,...,1.0,1.0,1.0,1.0,96.0,NaN,1758829034752,1.758829e+09,0.069,69
4,2025-09-25 15:37:14.752,0.092150,0.0,0.208309,0.324425,0.803366,0.0,0.117574,0.489226,1.779712,...,1.0,1.0,1.0,1.0,96.0,NaN,1758829034752,1.758829e+09,0.069,69
5,2025-09-25 15:37:14.752,0.092150,0.0,0.208309,0.324425,0.803366,0.0,0.117574,0.489226,1.779712,...,1.0,1.0,1.0,1.0,96.0,NaN,1758829034752,1.758829e+09,0.069,69
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11688,2025-09-25 15:37:59.415,0.417904,0.0,0.528940,0.740542,0.566095,0.0,0.133282,0.692467,1.410956,...,1.0,1.0,1.0,1.0,96.0,NaN,1758829079415,1.758829e+09,44.732,44732
11689,2025-09-25 15:37:59.415,0.417904,0.0,0.528940,0.740542,0.566095,0.0,0.133282,0.692467,1.410956,...,1.0,1.0,1.0,1.0,96.0,NaN,1758829079415,1.758829e+09,44.732,44732
11690,2025-09-25 15:37:59.415,0.410387,0.0,0.536043,0.747686,0.548732,0.0,0.161944,0.705485,1.410956,...,1.0,1.0,1.0,1.0,96.0,NaN,1758829079415,1.758829e+09,44.732,44732
11691,2025-09-25 15:37:59.415,0.410387,0.0,0.536043,0.747686,0.548732,0.0,0.161944,0.705485,1.410956,...,1.0,1.0,1.0,1.0,96.0,NaN,1758829079415,1.758829e+09,44.732,44732


,TimeStamp,unix_sec,unix_ms,rel_sec,rel_ms
1449,2025-09-25 15:37:19.622,1.758829e+09,1758829039622,4.939,4939
2677,2025-09-25 15:37:24.414,1.758829e+09,1758829044414,9.731,9731
9433,2025-09-25 15:37:50.698,1.758829e+09,1758829070698,36.015,36015
10773,2025-09-25 15:37:55.850,1.758829e+09,1758829075850,41.167,41167
11663,2025-09-25 15:37:59.308,1.758829e+09,1758829079308,44.625,44625


,TimeStamp,Delta_TP9,Delta_AF7,Delta_AF8,Delta_TP10,Theta_TP9,Theta_AF7,Theta_AF8,Theta_TP10,Alpha_TP9,...,HSI_TP9,HSI_AF7,HSI_AF8,HSI_TP10,Battery,Elements,unix_ms,unix_sec,rel_sec,rel_ms
1,2025-10-15 12:46:03.594,0.146566,0.382820,0.608571,0.462873,0.048322,0.135174,0.433407,0.311733,0.298287,...,2.0,1.0,1.0,2.0,20.0,NaN,1760546763594,1.760547e+09,0.085,85
2,2025-10-15 12:46:03.594,0.146566,0.382820,0.608571,0.462873,0.048322,0.135174,0.433407,0.311733,0.298287,...,2.0,1.0,1.0,2.0,20.0,NaN,1760546763594,1.760547e+09,0.085,85
3,2025-10-15 12:46:03.595,0.146566,0.382820,0.608571,0.462873,0.048322,0.135174,0.433407,0.311733,0.298287,...,2.0,1.0,1.0,2.0,20.0,NaN,1760546763595,1.760547e+09,0.086,86
4,2025-10-15 12:46:03.595,0.146566,0.382820,0.608571,0.462873,0.048322,0.135174,0.433407,0.311733,0.298287,...,2.0,1.0,1.0,2.0,20.0,NaN,1760546763595,1.760547e+09,0.086,86
5,2025-10-15 12:46:03.595,0.146566,0.382820,0.608571,0.462873,0.048322,0.135174,0.433407,0.311733,0.298287,...,2.0,1.0,1.0,2.0,20.0,NaN,1760546763595,1.760547e+09,0.086,86
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11456,2025-10-15 12:46:47.461,0.071092,0.088079,0.445837,0.467464,-0.182343,-0.151881,0.405305,0.058872,0.586831,...,1.0,1.0,1.0,1.0,25.0,NaN,1760546807461,1.760547e+09,43.952,43952
11457,2025-10-15 12:46:47.461,0.071092,0.088079,0.445837,0.467464,-0.182343,-0.151881,0.405305,0.058872,0.586831,...,1.0,1.0,1.0,1.0,25.0,NaN,1760546807461,1.760547e+09,43.952,43952
11458,2025-10-15 12:46:47.462,0.071092,0.088079,0.445837,0.467464,-0.182343,-0.151881,0.405305,0.058872,0.586831,...,1.0,1.0,1.0,1.0,25.0,NaN,1760546807462,1.760547e+09,43.953,43953
11459,2025-10-15 12:46:47.462,0.071092,0.088079,0.445837,0.467464,-0.182343,-0.151881,0.405305,0.058872,0.586831,...,1.0,1.0,1.0,1.0,25.0,NaN,1760546807462,1.760547e+09,43.953,43953


...

,TimeStamp,unix_sec,unix_ms,rel_sec,rel_ms
118,2025-10-30 16:48:59.995,1.761857e+09,1761857339995,0.123,123
355,2025-10-30 16:49:00.245,1.761857e+09,1761857340245,0.373,373
1136,2025-10-30 16:49:03.280,1.761857e+09,1761857343280,3.408,3408
3295,2025-10-30 16:49:11.787,1.761857e+09,1761857351787,11.915,11915
7638,2025-10-30 16:49:28.654,1.761857e+09,1761857368654,28.782,28782


,TimeStamp,Delta_TP9,Delta_AF7,Delta_AF8,Delta_TP10,Theta_TP9,Theta_AF7,Theta_AF8,Theta_TP10,Alpha_TP9,...,HSI_TP9,HSI_AF7,HSI_AF8,HSI_TP10,Battery,Elements,unix_ms,unix_sec,rel_sec,rel_ms
1,2025-11-22 10:28:20.737,0.526402,-0.388676,-0.253327,0.514473,0.117101,-0.284353,-0.319294,0.294521,0.500670,...,1.0,1.0,1.0,1.0,90.0,NaN,1763825300737,1.763825e+09,0.049,49
2,2025-11-22 10:28:20.738,0.526402,-0.388676,-0.253327,0.514473,0.117101,-0.284353,-0.319294,0.294521,0.500670,...,1.0,1.0,1.0,1.0,90.0,NaN,1763825300738,1.763825e+09,0.050,50
3,2025-11-22 10:28:20.738,0.526402,-0.388676,-0.253327,0.514473,0.117101,-0.284353,-0.319294,0.294521,0.500670,...,1.0,1.0,1.0,1.0,90.0,NaN,1763825300738,1.763825e+09,0.050,50
4,2025-11-22 10:28:20.738,0.526402,-0.388676,-0.253327,0.514473,0.117101,-0.284353,-0.319294,0.294521,0.500670,...,1.0,1.0,1.0,1.0,90.0,NaN,1763825300738,1.763825e+09,0.050,50
5,2025-11-22 10:28:20.738,0.526402,-0.388676,-0.253327,0.514473,0.117101,-0.284353,-0.319294,0.294521,0.500670,...,1.0,1.0,1.0,1.0,90.0,NaN,1763825300738,1.763825e+09,0.050,50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11756,2025-11-22 10:29:06.546,0.291423,-0.527020,-0.220989,-0.014362,0.119382,-0.541554,-0.100702,0.266718,0.675418,...,1.0,1.0,1.0,1.0,90.0,NaN,1763825346546,1.763825e+09,45.858,45858
11757,2025-11-22 10:29:06.546,0.291423,-0.527020,-0.220989,-0.014362,0.119382,-0.541554,-0.100702,0.266718,0.675418,...,1.0,1.0,1.0,1.0,90.0,NaN,1763825346546,1.763825e+09,45.858,45858
11758,2025-11-22 10:29:06.547,0.291423,-0.527020,-0.220989,-0.014362,0.119382,-0.541554,-0.100702,0.266718,0.675418,...,1.0,1.0,1.0,1.0,90.0,NaN,1763825346547,1.763825e+09,45.859,45859
11759,2025-11-22 10:29:06.547,0.291423,-0.527020,-0.220989,-0.014362,0.119382,-0.541554,-0.100702,0.266718,0.675418,...,1.0,1.0,1.0,1.0,90.0,NaN,1763825346547,1.763825e+09,45.859,45859


,TimeStamp,unix_sec,unix_ms,rel_sec,rel_ms


,TimeStamp,Delta_TP9,Delta_AF7,Delta_AF8,Delta_TP10,Theta_TP9,Theta_AF7,Theta_AF8,Theta_TP10,Alpha_TP9,...,HSI_TP9,HSI_AF7,HSI_AF8,HSI_TP10,Battery,Elements,unix_ms,unix_sec,rel_sec,rel_ms
1,2025-11-13 15:32:29.379,0.186764,-0.222491,-0.006990,0.404844,0.148234,-0.198548,-0.371817,0.241066,0.920284,...,1.0,1.0,1.0,1.0,100.0,NaN,1763065949379,1.763066e+09,0.071,71
2,2025-11-13 15:32:29.379,0.186764,-0.222491,-0.006990,0.404844,0.148234,-0.198548,-0.371817,0.241066,0.920284,...,1.0,1.0,1.0,1.0,100.0,NaN,1763065949379,1.763066e+09,0.071,71
3,2025-11-13 15:32:29.379,0.186764,-0.222491,-0.006990,0.404844,0.148234,-0.198548,-0.371817,0.241066,0.920284,...,1.0,1.0,1.0,1.0,100.0,NaN,1763065949379,1.763066e+09,0.071,71
4,2025-11-13 15:32:29.379,0.186764,-0.222491,-0.006990,0.404844,0.148234,-0.198548,-0.371817,0.241066,0.920284,...,1.0,1.0,1.0,1.0,100.0,NaN,1763065949379,1.763066e+09,0.071,71
5,2025-11-13 15:32:29.379,0.186764,-0.222491,-0.006990,0.404844,0.148234,-0.198548,-0.371817,0.241066,0.920284,...,1.0,1.0,1.0,1.0,100.0,NaN,1763065949379,1.763066e+09,0.071,71
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10511,2025-11-13 15:33:09.679,0.589256,1.014759,0.864074,0.322856,0.407358,0.667918,0.401683,0.720505,0.687661,...,1.0,1.0,1.0,1.0,100.0,NaN,1763065989679,1.763066e+09,40.371,40371
10512,2025-11-13 15:33:09.679,0.589256,1.014759,0.864074,0.322856,0.407358,0.667918,0.401683,0.720505,0.687661,...,1.0,1.0,1.0,1.0,100.0,NaN,1763065989679,1.763066e+09,40.371,40371
10513,2025-11-13 15:33:09.679,0.589256,1.014759,0.864074,0.322856,0.407358,0.667918,0.401683,0.720505,0.687661,...,1.0,1.0,1.0,1.0,100.0,NaN,1763065989679,1.763066e+09,40.371,40371
10514,2025-11-13 15:33:09.680,0.589256,1.014759,0.864074,0.322856,0.407358,0.667918,0.401683,0.720505,0.687661,...,1.0,1.0,1.0,1.0,100.0,NaN,1763065989680,1.763066e+09,40.372,40372


,TimeStamp,unix_sec,unix_ms,rel_sec,rel_ms
10200,2025-11-13 15:33:08.456,1.763066e+09,1763065988456,39.148,39148
10357,2025-11-13 15:33:09.067,1.763066e+09,1763065989067,39.759,39759
10462,2025-11-13 15:33:09.480,1.763066e+09,1763065989480,40.172,40172
